# Step 1: Setting up the environment

In [1]:
# Установка всех необходимых библиотек
print("Установка необходимых библиотек...")
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -Uq langchain-community # Обновление langchain-community для последних компонентов

print("Установка завершена.")

Установка необходимых библиотек...
Установка завершена.


Step 2: Loading the dataset

In [2]:
# --- Шаг 2: Загрузка датасета ---

# Убедимся, что 'datasets' обновлен (часто требуется для HuggingFaceDatasetLoader)
!pip install -Uq datasets

# Импорт HuggingFaceDatasetLoader
from langchain_community.document_loaders import HuggingFaceDatasetLoader

# Указываем имя датасета и столбец, который содержит основное содержимое
dataset_name = "databricks/databricks-dolly-15k"
# ИСПРАВЛЕНИЕ: ИЗМЕНИТЬ 'text' НА 'response'
page_content_column = "response" # <-- ИЗМЕНЕНО С "text" НА "response"

# Создаем экземпляр HuggingFaceDatasetLoader и загружаем данные как документы
print(f"\nЗагрузка датасета: {dataset_name}...")
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()
print(f"Загружено {len(data)} документов.")

# Опционально: Выводим первые 2 записи для проверки загрузки
print("\nПервые 2 записи загруженного датасета:")
print(data[:2])


Загрузка датасета: databricks/databricks-dolly-15k...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Загружено 15011 документов.

Первые 2 записи загруженного датасета:
[Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'category': 'closed_qa'}, page_content='"Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route."'), Document(metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'context': '', 'category': 'classification'}, page_content='"Tope"')]


# Step 3: Splitting Documents

In [3]:
# --- Шаг 3: Разделение документов ---
print("\nРазделение документов на чанки...")

# Импорт RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Создаем экземпляр RecursiveCharacterTextSplitter
# chunk_size=1000 означает, что каждый чанк будет стремиться быть не более 1000 символов.
# chunk_overlap=150 означает, что каждые 150 символов будут повторяться между соседними чанками.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# Разбиваем загруженные документы
docs = text_splitter.split_documents(data) # 'data' - это список документов, загруженных на предыдущем шаге

print(f"Исходных документов: {len(data)}")
print(f"Документы разбиты на {len(docs)} чанков.")

# Опционально: Выводим первый чанк документа для проверки
print("\nПервый чанк документа:")
print(docs[0])


Разделение документов на чанки...
Исходных документов: 15011
Документы разбиты на 16744 чанков.

Первый чанк документа:
page_content='"Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route."' metadata={'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'category': 'closed_qa'}


# Step 4: Embed the text

In [4]:
# --- Шаг 4: Эмбеддинг текста ---
print("\nСоздание эмбеддингов для текстовых чанков...")

# Импорт HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Определяем путь к модели (выбираем легкую, но эффективную модель для эмбеддингов)
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Определяем конфигурации модели. 'device':'cpu' означает, что модель будет работать на CPU.
# Если у вас есть GPU (например, в Colab с включенным GPU), можете попробовать 'cuda'.
model_kwargs = {'device':'cpu'}

# Определяем опции кодирования. 'normalize_embeddings': False означает, что векторы не будут нормализованы до единичной длины.
# Для некоторых метрик сходства (например, косинусного сходства) нормализация может быть полезна,
# но для FAISS это не всегда строго необходимо, и модель может быть обучена без нее.
encode_kwargs = {'normalize_embeddings': False}

# Инициализируем HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
  model_name=modelPath,
  model_kwargs=model_kwargs,
  encode_kwargs=encode_kwargs
)

print(f"Модель эмбеддингов '{modelPath}' загружена.")

# (Опционально) Тестирование создания эмбеддинга
# Это покажет, как выглядит вектор для простого текста.
print("\nТестирование создания эмбеддинга для 'This is a test document.':")
text = "This is a test document."
query_result = embeddings.embed_query(text)
print(f"Длина эмбеддинга: {len(query_result)}")
print(f"Первые 3 элемента эмбеддинга: {query_result[:3]}")


Создание эмбеддингов для текстовых чанков...


/tmp/ipython-input-4-1322918127.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Модель эмбеддингов 'sentence-transformers/all-MiniLM-l6-v2' загружена.

Тестирование создания эмбеддинга для 'This is a test document.':
Длина эмбеддинга: 384
Первые 3 элемента эмбеддинга: [-0.03833857178688049, 0.1234646737575531, -0.028642933815717697]


# Step 5: Create a vector store

In [5]:
# --- Шаг 5: Создание векторного хранилища ---
print("\nСоздание векторного хранилища FAISS...")

# Импорт FAISS
from langchain_community.vectorstores import FAISS

# Создаем векторное хранилище FAISS из списка документов (чанков) и нашей модели эмбеддингов.
# FAISS будет использовать модель 'embeddings' для преобразования каждого 'doc' в вектор,
# а затем индексировать эти векторы для быстрого поиска.
db = FAISS.from_documents(docs, embeddings)

print("Векторное хранилище FAISS успешно создано и проиндексировано.")
print("Теперь можно выполнять быстрый семантический поиск по нашим документам.")

# (Опционально) Тестирование поиска по сходству
# Попробуем найти документы, похожие на наш тестовый вопрос.
query = "What is the capital of France?"
found_docs = db.similarity_search(query)
print(f"\nРезультаты поиска для '{query}':")
for i, doc in enumerate(found_docs):
     print(f"  Документ {i+1}:")
     print(f"    Content: {doc.page_content[:200]}...") # Выводим часть содержимого
     print(f"    Metadata: {doc.metadata}")
     print("-" * 20)


Создание векторного хранилища FAISS...


KeyboardInterrupt: 

# Step 6: Prepare the LLM model

In [ ]:
# --- Шаг 6: Подготовка модели LLM ---
print("\nПодготовка LLM модели для вопросно-ответной системы...")

# Импорт необходимых классов из transformers и langchain
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain_community.llms import HuggingFacePipeline # Обновленный импорт для HuggingFacePipeline

# Загрузка токенизатора и модели для задачи Question Answering
model_name = "Intel/dynamic_tinybert"

print(f"Загрузка токенизатора: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)
print("Токенизатор загружен.")

print(f"Загрузка модели Question Answering: {model_name}...")
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
print("Модель загружена.")

# Создание пайплайна Question Answering
# Этот пайплайн будет принимать вопрос и контекст, и извлекать ответ из контекста.
print("\nСоздание пайплайна Question Answering...")
qa_pipeline = pipeline(
  "question-answering",
  model=model,        # Используем загруженную модель
  tokenizer=tokenizer,  # Используем загруженный токенизатор
  return_tensors='pt',  # Возвращаем PyTorch тензоры
  # Дополнительные аргументы, которые могут быть важны для производительности/качества:
  # max_length - максимальная длина генерируемого ответа
  # top_k - количество лучших ответов для рассмотрения (для QA)
  # handle_impossible_answer - обрабатывать ли ситуации, когда ответ не найден в контексте
)
print("Пайплайн Question Answering создан.")

# Создание обертки Langchain для пайплайна HuggingFace
# Это позволяет Langchain взаимодействовать с нашим пайплайном как с LLM.
print("\nСоздание Langchain Pipeline Wrapper...")
llm = HuggingFacePipeline(
  pipeline=qa_pipeline,
  # model_kwargs могут передавать аргументы непосредственно в модель, но для QA пайплайна
  # они часто передаются через создание самого пайплайна или в 'generate_kwargs'.
  # temperature здесь не актуальна, так как это не генеративная модель в чистом виде,
  # а экстрактивная QA. max_length уже задан в пайплайне.
  # Поэтому для этой конкретной модели Intel/dynamic_tinybert, model_kwargs можно убрать или изменить.
  # Для других LLM (например, для генерации текста) temperature и max_length были бы очень важны.
  # Здесь оставим их, чтобы соответствовать инструкции, но понимай их смысл.
  model_kwargs={"temperature": 0.0, "max_length": 512}, # temperature 0.0 для детерминированного ответа в QA
)
print("Langchain Pipeline Wrapper для LLM создан.")

In [ ]:
# --- Промежуточная проверка qa_pipeline напрямую ---
print("\nПроверка qa_pipeline напрямую для отладки...")

test_question_direct = "What is the capital of France?"
# Вы можете взять реальный извлеченный контекст из предыдущего вывода
# Например, для "What is the capital of France?" один из документов был:
test_context_direct = "The capital of France is Paris" # Используем наиболее релевантный контекст

# Попробуем запустить qa_pipeline напрямую
try:
    direct_qa_result = qa_pipeline([{'question': test_question_direct, 'context': test_context_direct}])
    print(f"Прямой результат qa_pipeline для '{test_question_direct}': {direct_qa_result}")
    print(f"Тип прямого результата: {type(direct_qa_result)}")
    if direct_qa_result and len(direct_qa_result) > 0 and 'answer' in direct_qa_result[0]:
        direct_answer = direct_qa_result[0]['answer']
        print(f"Извлеченный ответ (прямой): '{direct_answer}'")
        print(f"Тип извлеченного ответа (прямой): {type(direct_answer)}")
    else:
        print("WARN: Прямой запуск qa_pipeline не вернул ожидаемый формат ответа.")

except Exception as e:
    print(f"ОШИБКА при прямом запуске qa_pipeline: {e}")

# Теперь попробуем с запросом, который не имеет прямого ответа в контексте, или контекст нерелевантен
test_question_no_answer = "What is the purpose of art?"
test_context_no_answer = "Art is a diverse range of human activities in creating visual, auditory or performing artifacts (artworks), expressing the author's imaginative or technical skill, intended to be appreciated for their beauty or emotional power." # Пример контекста, где прямого ответа нет

try:
    direct_qa_result_no_answer = qa_pipeline([{'question': test_question_no_answer, 'context': test_context_no_answer}])
    print(f"\nПрямой результат qa_pipeline для '{test_question_no_answer}': {direct_qa_result_no_answer}")
    print(f"Тип прямого результата (нет ответа): {type(direct_qa_result_no_answer)}")
    if direct_qa_result_no_answer and len(direct_qa_result_no_answer) > 0 and 'answer' in direct_qa_result_no_answer[0]:
        direct_answer_no_answer = direct_qa_result_no_answer[0]['answer']
        print(f"Извлеченный ответ (нет ответа): '{direct_answer_no_answer}'")
        print(f"Тип извлеченного ответа (нет ответа): {type(direct_answer_no_answer)}")
    else:
        print("WARN: Прямой запуск qa_pipeline не вернул ожидаемый формат ответа для запроса без прямого ответа.")

except Exception as e:
    print(f"ОШИБКА при прямом запуске qa_pipeline (нет ответа): {e}")

# Step 7: Build the Retrieval QA Chain

In [ ]:
# --- Шаг 7 (ФИНАЛЬНОЕ ИСПРАВЛЕНИЕ - ВЕРСИЯ 5 LCEL): Построение цепочки Retrieval QA Chain ---
print("\nПостроение цепочки Retrieval QA Chain (ФИНАЛЬНАЯ ВЕРСИЯ 5 LCEL)...")

from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Наш ретривер уже готов
retriever = db.as_retriever(search_kwargs={"k": 4})
print("Ретривер готов.")

# Функция для форматирования извлеченных документов в одну строку
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Новая именованная функция для обработки QA логики и отладки
def process_qa_with_debug(input_dict):
    """
    Обрабатывает вопрос и контекст, вызывает qa_pipeline и возвращает ответ.
    Включает отладочные принты.
    """
    question = input_dict['question']
    context = input_dict['context']

    print(f"\n--- DEBUG: Вход в process_qa_with_debug ---")
    print(f"DEBUG: Question: {question}")
    print(f"DEBUG: Context (первые 200 симв.): {context[:200]}...")

    qa_result = qa_pipeline([{'question': question, 'context': context}])

    print(f"DEBUG: Результат qa_pipeline: {qa_result}")
    print(f"DEBUG: Тип результата qa_pipeline: {type(qa_result)}")
    print(f"DEBUG: Первый элемент результата qa_pipeline: {qa_result[0] if qa_result else 'Пусто'}")
    print(f"DEBUG: Тип первого элемента: {type(qa_result[0]) if qa_result else 'Пусто'}")

    # Извлекаем ответ. Обратите внимание: экстрактивные модели Q&A всегда должны возвращать ответ.
    # Если qa_result пустой или не содержит 'answer', это проблема модели/пайплайна.
    answer = ""
    if qa_result and isinstance(qa_result, list) and len(qa_result) > 0 and 'answer' in qa_result[0]:
        answer = qa_result[0]['answer']
    else:
        answer = "Извините, не удалось найти ответ на основе предоставленного контекста."
        print(f"WARNING: 'answer' не найден в результате qa_pipeline: {qa_result}")

    print(f"DEBUG: Извлеченный ответ: '{answer}'")
    print(f"DEBUG: Тип извлеченного ответа: {type(answer)}")
    print(f"--- DEBUG: Выход из process_qa_with_debug ---")

    return answer # Обязательно возвращаем только строку!

# ФИНАЛЬНАЯ ЦЕПОЧКА LCEL для экстрактивной QA модели:
qa_chain = (
    {
        "context": retriever | format_docs,  # Извлекаем и форматируем контекст
        "question": RunnablePassthrough()    # Передаем оригинальный вопрос
    }
    # Теперь вызываем именованную функцию через RunnableLambda
    | RunnableLambda(process_qa_with_debug)
)

print("Цепочка RAG (через LCEL с именованной функцией для QA и отладкой) успешно построена.")

# Step 8: Test your RAG system

In [ ]:
# --- Шаг 8 (ОКОНЧАТЕЛЬНОЕ ИСПРАВЛЕНИЕ): Тестирование вашей RAG-системы ---
print("\nТестирование RAG-системы (финальная попытка)...")

questions_to_test = [
    "What is cheesemaking?",
    "What is the capital of France?",
    "How does photosynthesis work?",
    "Who was Abraham Lincoln?",
    "What are the benefits of meditation?",
    "Tell me about machine learning.",
    "What is the purpose of art?"
]

for q in questions_to_test:
    print(f"\nЗапрос: '{q}'")
    try:
        # Сначала извлекаем контекст для отладки
        docs_for_debug = retriever.invoke(q)
        print("Извлеченный контекст для отладки:")
        for i, d in enumerate(docs_for_debug):
            print(f"  Документ {i+1}:")
            print(f"    Content: {d.page_content[:400]}...")
            if d.metadata:
                print(f"    Metadata: {d.metadata}")
            print("-" * 20)
        print("-" * 30)

        # *** ИСПРАВЛЕНИЕ: Теперь qa_chain напрямую возвращает строку с ответом ***
        answer = qa_chain.invoke({"question": q}) # Вызываем LCEL-цепочку
        print(f"Ответ RAG-системы: {answer}")

    except Exception as e:
        print(f"Произошла ошибка при выполнении запроса: {e}")
        # Повторно извлекаем контекст для отладки, если произошла ошибка
        try:
            docs_for_debug = retriever.invoke(q)
            print("Извлеченный контекст для отладки:")
            for i, d in enumerate(docs_for_debug):
                print(f"  Документ {i+1}:")
                print(f"    Content: {d.page_content[:400]}...")
                if d.metadata:
                    print(f"    Metadata: {d.metadata}")
                print("-" * 20)
            print("-" * 30)
        except Exception as debug_e:
            print(f"Не удалось извлечь контекст для отладки: {debug_e}")